In [14]:
# root folder C:/Users/SergeyRyzhov/
import pandas as pd
import re
import numpy as np
#loadinf domain operations data
resets = pd.read_csv("C:/Users/SergeyRyzhov/Downloads/BQ data/domain_operation_raw.csv")
#picking out template and acc ids from operation data
resets.insert (5,"Account",0)
resets.insert (6,"Template",0)
resets.reset_index()
for index,row in resets.iterrows():
    if row["operation_type"] == "DemoAccountsDumpRestore":
        resets.at[index,"operation_type"] = "Reset to zero"
    else:
        resets.at[index,"operation_type"] = "Reset dates"
    data_string = str(row["operation_data"])
    x = re.search('"templateId":\d+', data_string)
    x_id_text = x.group()
    clean_templ_id = re.search('\d+', x_id_text)
    resets.at[index,"Template"] = clean_templ_id.group()
    y = re.search('"demoId":\d+|"demoAccountId":\d+', data_string)
    if y == None: #there are a few null values somehow so need an exception
         resets.at[index,"Account"] = "0"
    else:
        y_id_text = y.group()
        clean_acc_id = re.search('\d+', y_id_text)
        resets.at[index,"Account"] = clean_acc_id.group()
#rename what we need
resets['Template'].mask(resets['Template'] == '3075994','M&PS',inplace = True)
resets['Template'].mask(resets['Template'] == '4365873','MDT',inplace = True)
#change data type in resets
resets['Account'] = resets['Account'].astype('int64')
# adding owner IDs and account start dates from the demo acc table
owners = pd.read_csv ("C:/Users/SergeyRyzhov/Downloads/BQ data/demo_account_raw.csv")
resets = resets.merge (owners, how="left", left_on='Account',right_on='account_id')
#adding user names where possible, first demo acc holders
users = pd.read_excel("C:/Users/SergeyRyzhov/Downloads/Misc data/Users Wrike.xlsx")
users.rename(columns = {'Name':'Owner Name'},inplace=True)
users.drop(['User Status','User Role'], axis=1, inplace=True)
resets = resets.merge (users,how='left',left_on='DemoOwner',right_on='ID')
resets.drop('ID', axis=1, inplace=True)
users.rename(columns = {'Owner Name':'Resetter Name'},inplace=True)
resets = resets.merge (users,how='left',left_on='ResetterID',right_on='ID')
resets.drop('ID', axis=1, inplace=True)
#add more account data
accounts = pd.read_csv('C:/Users/SergeyRyzhov/Downloads/BQ data/all_accounts_raw.csv')
resets = resets.merge(accounts, how = 'left', left_on = 'Account', right_on = 'AccountID')
#convert the dates
resets['Datetime']=pd.to_datetime(resets['Datetime']).dt.date
resets['DemoStart']=pd.to_datetime(resets['DemoStart']).dt.date
resets['CreatedOn']=pd.to_datetime(resets['CreatedOn']).dt.date
resets['DeletedOn']=pd.to_datetime(resets['DeletedOn']).dt.date
#fixing the data types
resets['DemoOwner'].fillna(0,inplace=True)
resets['AccountID'].fillna(0,inplace=True)
resets['DemoOwner'] = resets['DemoOwner'].astype('int64')
resets['AccountID'] = resets['AccountID'].astype('int64')
resets.replace(0,np.nan)
#final cleanup
resets.drop('operation_data', axis=1, inplace=True)
resets.drop('account_id', axis=1, inplace=True)
#change reset to zero when it's actually an acc creation
for index,row in resets.iterrows():    
    if row['operation_type'] == 'Reset to zero' and row['Datetime'] == row['CreatedOn']:
        resets.at[index,"operation_type"] = "Create account"
resets.to_csv('demo_acc_resets_wrangled.csv')

C:\Users\SergeyRyzhov\Downloads\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [15]:
resets.sample(10)

,OperationID,operation_type,Datetime,ResetterID,Account,Template,DemoOwner,DemoStart,Owner Name,Resetter Name,AccountID,CreatedOn,DeletedOn
221,237288,Reset to zero,2019-11-22,7062023,3258975,M&PS,7062023,2019-11-18,NaN,NaN,0,NaT,NaT
5512,368054,Reset dates,2022-08-22,9764951,5547684,MDT,9764951,2022-05-17,Steve Molloy,Steve Molloy,5547684,2022-05-17,NaT
1000,267963,Reset to zero,2020-06-15,2998616,3402284,M&PS,2998616,2020-01-14,NaN,NaN,0,NaT,NaT
4144,348463,Reset dates,2022-03-08,6233902,3178775,M&PS,6233902,2019-10-18,Ryo Shinohara,Ryo Shinohara,0,NaT,NaT
3448,337756,Create account,2021-12-13,6131514,5229246,MDT,12162766,2021-12-13,Jose Cabada,Cronan Guinan-Menton,5229246,2021-12-13,NaT
408,243752,Reset dates,2020-01-15,2189832,3178783,M&PS,2189832,2019-10-18,NaN,NaN,0,NaT,NaT
4380,351641,Reset dates,2022-04-04,12376467,5290340,M&PS,12376467,2022-01-11,Cristina Viale,Cristina Viale,5290340,2022-01-11,NaT
2630,320704,Reset dates,2021-07-21,3016557,3178538,M&PS,3016557,2019-10-18,Kaan Aktuglu,Kaan Aktuglu,0,NaT,NaT
2923,328866,Reset dates,2021-09-22,10651093,4706966,MDT,10651093,2021-06-07,Lisa Starbuck,Lisa Starbuck,4706966,2021-06-07,NaT
3216,334249,Reset dates,2021-11-08,1787242,5071896,MDT,1787242,2021-10-20,Sergey Ryzhov,Sergey Ryzhov,5071896,2021-10-20,NaT
